import pandas as pd
import numpy as np
import unicodedata
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from tqdm import tqdm
import time
import gc
from nltk.stem.snowball import SpanishStemmer

# -------------------------------------
# CONFIG
# -------------------------------------
RUTA_BARBADOS = r'C:\Users\dreynosoh\Downloads\Prueba Publicaciones.xlsx'
RUTA_CATALOGO = r'C:\Users\dreynosoh\Downloads\Catalogo Truper agosto 25_wcm.xlsx'
SALIDA_FINAL = r'C:\Users\dreynosoh\Downloads\publicaciones_eibel2.0.xlsx'

BLOQUE = 10000  # Tamaño del bloque (ajustar según RAM disponible)

# -------------------------------------
# DESCARGAR STOPWORDS
# -------------------------------------
nltk.download('stopwords')
stopwords_es = set(stopwords.words("spanish"))

# -------------------------------------
# FUNCIONES DE PREPROCESAMIENTO
# -------------------------------------
def normalizar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = texto.lower()
    texto = re.sub(r'[^a-z0-9\s/\-]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def tokenize(texto):
    tokens = texto.split()
    unidades = {'hp', 'kg', 'ml'}
    return {word for word in tokens if word not in stopwords_es or word in unidades}

def stem_tokens(text):
    stemmer = SpanishStemmer()
    return {stemmer.stem(t) for t in text.split()}

# -------------------------------------
# CARGA DE DATOS
# -------------------------------------
barbados = pd.read_excel(RUTA_BARBADOS)
catalogo = pd.read_excel(RUTA_CATALOGO)

# -------------------------------------
# NORMALIZAR TEXTOS
# -------------------------------------
barbados['Titulo_Publicacion_normalizado'] = barbados['Titulo_Publicacion'].apply(normalizar_texto).astype(str)
catalogo['descripcion_normalizada'] = catalogo['descripción'].apply(normalizar_texto).astype(str)
catalogo['clave'] = catalogo['clave'].fillna("").astype(str)
catalogo['Codigo'] = catalogo['Codigo'].fillna("").astype(str)

# -------------------------------------
# TF-IDF VECTORIZE EL CATALOGO
# -------------------------------------
vectorizer = TfidfVectorizer()
tfidf_catalog = vectorizer.fit_transform(catalogo['descripcion_normalizada'])

# Convertimos catalogo a lista de diccionarios
catalog_list = catalogo[['descripcion_normalizada', 'clave', 'Codigo']].to_dict(orient='records')

# -------------------------------------
# INICIALIZAR RESULTADOS
# -------------------------------------
resultados = []

# -------------------------------------
# PROCESAMIENTO EN BLOQUES CON TIEMPOS
# -------------------------------------
n_total = len(barbados)
n_bloques = (n_total // BLOQUE) + 1

print(f"🔄 Procesando {n_total:,} filas en {n_bloques} bloques...\\n")
start_time_total = time.time()

for i in tqdm(range(n_bloques), desc="🧠 Procesando bloques"):
    bloque_start_time = time.time()

    ini = i * BLOQUE
    fin = min((i + 1) * BLOQUE, n_total)
    bloque_df = barbados.iloc[ini:fin].copy()

    textos_bloque = bloque_df['Titulo_Publicacion_normalizado'].tolist()
    tfidf_bloque = vectorizer.transform(textos_bloque)
    similarities = cosine_similarity(tfidf_bloque, tfidf_catalog)

    for j, row in enumerate(bloque_df.itertuples(index=False)):
        texto_norm = row.Titulo_Publicacion_normalizado
        tokens = tokenize(texto_norm)
        texto_sin_slash = texto_norm.replace("/", "")
    
        match_found = False
        palabra_match = None
        clave_match = None
        codigo_match = None
        tipo_match = None
        palabras_comunes = []
        best_score = 0.0
    
        # --------------------------------
        # 1️⃣ MATCH EXACTO: CÓDIGO (solo si tiene 5 o 6 dígitos)
        # --------------------------------
        for word in texto_sin_slash.split():
            if word.isdigit() and 5 <= len(word) <= 6:
                for item in catalog_list:
                    if word == item["Codigo"]:
                        codigo_match = item["Codigo"]
                        clave_match = item["clave"]
                        tipo_match = "codigo"
                        palabra_match = word
                        match_found = True
                        break
            if match_found:
                break
    
        # --------------------------------
        # 2️⃣ MATCH EXACTO: CLAVE (si no hubo match por código)
        # --------------------------------
        if not match_found:
            for word in texto_sin_slash.split():
                for item in catalog_list:
                    if word.upper() == item["clave"].replace("/", "").upper():
                        clave_match = item["clave"]
                        codigo_match = item["Codigo"]
                        tipo_match = "clave"
                        palabra_match = word
                        match_found = True
                        break
                if match_found:
                    break
    
        # --------------------------------
        # 3️⃣ MATCH POR COSINE SIMILARITY (si no hubo match anterior)
        # --------------------------------
        if not match_found:
            sims = similarities[j]
            best_idx = np.argmax(sims)
            best_score = sims[best_idx]
            best_item = catalog_list[best_idx]
        
            clave_match = best_item["clave"]
            codigo_match = best_item["Codigo"]
            tipo_match = "cosine"
        
            # Tokenización con stemming para detectar más coincidencias
            tokens_stem = stem_tokens(texto_norm)
            descripcion_stem = stem_tokens(best_item["descripcion_normalizada"])
        
            # Buscar coincidencias flexibles (exactas o parciales)
            palabras_comunes = []
            for w1 in tokens_stem:
                for w2 in descripcion_stem:
                    # Coincidencia directa o substring parcial
                    if w1 == w2 or w1 in w2 or w2 in w1:
                        palabras_comunes.append(w1)
        
            palabras_comunes = list(set(palabras_comunes))  # eliminar duplicados
    
        # --------------------------------
        # Guardar resultado
        # --------------------------------
        resultados.append({
            "Titulo_Publicacion": row.Titulo_Publicacion,
            "Titulo_Publicacion_normalizado": texto_norm,
            "clave_asignada": clave_match,
            "codigo_asignado": codigo_match,
            "tipo_match": tipo_match,
            "score_cosine": round(best_score, 4) if not match_found else 1.0,
            "palabra_match": palabra_match,
            "palabras_comunes": ", ".join(palabras_comunes)
        })

    # 🕒 TIEMPO POR BLOQUE
    bloque_duracion = time.time() - bloque_start_time
    tiempo_total_actual = time.time() - start_time_total
    tiempo_promedio = tiempo_total_actual / (i + 1)
    bloques_restantes = n_bloques - (i + 1)
    estimado_restante = bloques_restantes * tiempo_promedio

    print(f"\\n🕒 Bloque {i+1}/{n_bloques} procesado en {bloque_duracion:.1f}s")
    print(f"⏳ Tiempo total: {tiempo_total_actual/60:.1f} min")
    print(f"🔮 Estimado restante: {estimado_restante/60:.1f} min\\n")

    # Limpiar memoria
    del tfidf_bloque, similarities
    gc.collect()

# -------------------------------------
# GUARDAR RESULTADOS
# -------------------------------------
final_df = pd.DataFrame(resultados)
final_df.to_excel(SALIDA_FINAL, index=False)
tiempo_total_final = time.time() - start_time_total

print(f"\\n✅ Proceso COMPLETADO en {tiempo_total_final/60:.1f} minutos.")
print(f"📁 Archivo guardado en: {SALIDA_FINAL}")

In [2]:
import pandas as pd
import numpy as np
import unicodedata
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from tqdm import tqdm
import time
import gc
from nltk.stem.snowball import SpanishStemmer

# -------------------------------------
# CONFIG
# -------------------------------------
RUTA_BARBADOS = r'C:\Users\dreynosoh\Downloads\Prueba Publicaciones.xlsx'
RUTA_CATALOGO = r'C:\Users\dreynosoh\Downloads\Catalogo Truper agosto 25_wcm.xlsx'
SALIDA_FINAL = r'C:\Users\dreynosoh\Downloads\publicaciones_eibel2.0.xlsx'

BLOQUE = 10000  # Tamaño del bloque (ajustar según RAM disponible)

# -------------------------------------
# DESCARGAR STOPWORDS
# -------------------------------------
nltk.download('stopwords')
stopwords_es = set(stopwords.words("spanish"))

# -------------------------------------
# FUNCIONES DE PREPROCESAMIENTO
# -------------------------------------
def normalizar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = texto.lower()
    texto = re.sub(r'[^a-z0-9\s/\-]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def tokenize(texto):
    tokens = texto.split()
    unidades = {'hp', 'kg', 'ml'}
    return {word for word in tokens if word not in stopwords_es or word in unidades}

def stem_tokens(text):
    stemmer = SpanishStemmer()
    return {stemmer.stem(t) for t in text.split()}

# -------------------------------------
# CARGA DE DATOS
# -------------------------------------
barbados = pd.read_excel(RUTA_BARBADOS)
catalogo = pd.read_excel(RUTA_CATALOGO)

# -------------------------------------
# NORMALIZAR TEXTOS
# -------------------------------------
barbados['Titulo_Publicacion_normalizado'] = barbados['Titulo_Publicacion'].apply(normalizar_texto).astype(str)
catalogo['descripcion_normalizada'] = catalogo['descripción'].apply(normalizar_texto).astype(str)
catalogo['clave'] = catalogo['clave'].fillna("").astype(str)
catalogo['Codigo'] = catalogo['Codigo'].fillna("").astype(str)

# -------------------------------------
# TF-IDF VECTORIZE EL CATALOGO
# -------------------------------------
vectorizer = TfidfVectorizer()
tfidf_catalog = vectorizer.fit_transform(catalogo['descripcion_normalizada'])

# Convertimos catalogo a lista de diccionarios
catalog_list = catalogo[['descripcion_normalizada', 'clave', 'Codigo']].to_dict(orient='records')

# -------------------------------------
# INICIALIZAR RESULTADOS
# -------------------------------------
resultados = []

# -------------------------------------
# PROCESAMIENTO EN BLOQUES CON TIEMPOS
# -------------------------------------
n_total = len(barbados)
n_bloques = (n_total // BLOQUE) + 1

print(f"🔄 Procesando {n_total:,} filas en {n_bloques} bloques...\n")
start_time_total = time.time()

for i in tqdm(range(n_bloques), desc="🧠 Procesando bloques"):
    bloque_start_time = time.time()

    ini = i * BLOQUE
    fin = min((i + 1) * BLOQUE, n_total)
    bloque_df = barbados.iloc[ini:fin].copy()

    textos_bloque = bloque_df['Titulo_Publicacion_normalizado'].tolist()
    tfidf_bloque = vectorizer.transform(textos_bloque)
    similarities = cosine_similarity(tfidf_bloque, tfidf_catalog)

    for j, row in enumerate(bloque_df.itertuples(index=False)):
        texto_norm = row.Titulo_Publicacion_normalizado
        tokens = tokenize(texto_norm)
        texto_sin_slash = texto_norm.replace("/", "")
    
        match_found = False
        palabra_match = None
        clave_match = None
        codigo_match = None
        tipo_match = None
        palabras_comunes = []
        best_score = 0.0
    
        # --------------------------------
        # 1️⃣ MATCH EXACTO: CÓDIGO (solo si tiene 5 o 6 dígitos)
        # --------------------------------
        for word in texto_sin_slash.split():
            if word.isdigit() and 5 <= len(word) <= 6:
                for item in catalog_list:
                    if word == item["Codigo"]:
                        codigo_match = item["Codigo"]
                        clave_match = item["clave"]
                        tipo_match = "codigo"
                        palabra_match = word
                        match_found = True
                        break
            if match_found:
                break
    
        # --------------------------------
        # 2️⃣ MATCH EXACTO: CLAVE (si no hubo match por código)
        # --------------------------------
        if not match_found:
            for word in texto_sin_slash.split():
                # ✅ SOLO CONSIDERAR PALABRAS CON MÁS DE 2 CARACTERES
                if len(word) > 2:
                    for item in catalog_list:
                        if word.upper() == item["clave"].replace("/", "").upper():
                            clave_match = item["clave"]
                            codigo_match = item["Codigo"]
                            tipo_match = "clave"
                            palabra_match = word
                            match_found = True
                            break
                    if match_found:
                        break
    
        # --------------------------------
        # 3️⃣ MATCH POR COSINE SIMILARITY (si no hubo match anterior)
        # --------------------------------
        if not match_found:
            sims = similarities[j]
            best_idx = np.argmax(sims)
            best_score = sims[best_idx]
            best_item = catalog_list[best_idx]
        
            clave_match = best_item["clave"]
            codigo_match = best_item["Codigo"]
            tipo_match = "cosine"
        
            # Tokenización con stemming para detectar más coincidencias
            tokens_stem = stem_tokens(texto_norm)
            descripcion_stem = stem_tokens(best_item["descripcion_normalizada"])
        
            # Buscar coincidencias flexibles (exactas o parciales)
            palabras_comunes = []
            for w1 in tokens_stem:
                for w2 in descripcion_stem:
                    # Coincidencia directa o substring parcial
                    if w1 == w2 or w1 in w2 or w2 in w1:
                        palabras_comunes.append(w1)
        
            palabras_comunes = list(set(palabras_comunes))  # eliminar duplicados
    
        # --------------------------------
        # Guardar resultado
        # --------------------------------
        resultados.append({
            "Titulo_Publicacion": row.Titulo_Publicacion,
            "Titulo_Publicacion_normalizado": texto_norm,
            "clave_asignada": clave_match,
            "codigo_asignado": codigo_match,
            "tipo_match": tipo_match,
            "score_cosine": round(best_score, 4) if not match_found else 1.0,
            "palabra_match": palabra_match,
            "palabras_comunes": ", ".join(palabras_comunes)
        })

    # 🕒 TIEMPO POR BLOQUE
    bloque_duracion = time.time() - bloque_start_time
    tiempo_total_actual = time.time() - start_time_total
    tiempo_promedio = tiempo_total_actual / (i + 1)
    bloques_restantes = n_bloques - (i + 1)
    estimado_restante = bloques_restantes * tiempo_promedio

    print(f"\n🕒 Bloque {i+1}/{n_bloques} procesado en {bloque_duracion:.1f}s")
    print(f"⏳ Tiempo total: {tiempo_total_actual/60:.1f} min")
    print(f"🔮 Estimado restante: {estimado_restante/60:.1f} min\n")

    # Limpiar memoria
    del tfidf_bloque, similarities
    gc.collect()

# -------------------------------------
# GUARDAR RESULTADOS
# -------------------------------------
final_df = pd.DataFrame(resultados)
final_df.to_excel(SALIDA_FINAL, index=False)
tiempo_total_final = time.time() - start_time_total

print(f"\n✅ Proceso COMPLETADO en {tiempo_total_final/60:.1f} minutos.")
print(f"📁 Archivo guardado en: {SALIDA_FINAL}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dreynosoh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🔄 Procesando 14,571 filas en 2 bloques...



🧠 Procesando bloques:  50%|█████     | 1/2 [04:23<04:23, 263.71s/it]


🕒 Bloque 1/2 procesado en 262.6s
⏳ Tiempo total: 4.4 min
🔮 Estimado restante: 4.4 min



🧠 Procesando bloques: 100%|██████████| 2/2 [06:53<00:00, 206.67s/it]


🕒 Bloque 2/2 procesado en 149.4s
⏳ Tiempo total: 6.9 min
🔮 Estimado restante: 0.0 min




✅ Proceso COMPLETADO en 7.0 minutos.
📁 Archivo guardado en: C:\Users\dreynosoh\Downloads\publicaciones_eibel2.0.xlsx
